In [8]:
# ==============================
# 1. Importações
# ==============================
from ecmwf.opendata import Client
from datetime import datetime, timedelta
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.feature import NaturalEarthFeature
import numpy as np
import pandas as pd
import os, warnings
from matplotlib.colors import ListedColormap, BoundaryNorm

warnings.filterwarnings("ignore", category=FutureWarning)

# ==============================
# 2. Nomes dos dias em português
# ==============================
dias_semana_pt = {
    "Monday": "segunda-feira",
    "Tuesday": "terça-feira",
    "Wednesday": "quarta-feira",
    "Thursday": "quinta-feira",
    "Friday": "sexta-feira",
    "Saturday": "sábado",
    "Sunday": "domingo",
}

# ==============================
# 3. Parâmetros
# ==============================
nivels = [0,0.5, 2, 5, 10, 20, 30, 40, 50, 75, 100, 150, 200, 300]
cores = [
    "#FFFFFF","#dedede","#919191","#63ff83","#00C000","#0f13f6",
    "#FFFF00","#FFA500","#FF0000","#C00000","#800000","#660066","#330033"
]
color_map = ListedColormap(cores)
norma = BoundaryNorm(nivels, color_map.N)
tick_locs = [(nivels[i]+nivels[i+1])/2 for i in range(len(nivels)-1)]
tick_labels = [f"{nivels[i]}–{nivels[i+1]}" for i in range(len(nivels)-1)]
tick_labels[-1] = f">{nivels[-2]}"
extent = [-85, -30, -35, 10]
out_dir = os.getcwd()
os.makedirs(out_dir, exist_ok=True)

# ==============================
# 4. Download ECMWF HRES travado para 00Z (15 dias)
# ==============================
client = Client(source="ecmwf")

# Hora atual no Brasil (UTC-3)
now_br = datetime.utcnow() - timedelta(hours=3)
date_run = now_br.date()
run_date_str = date_run.strftime("%Y%m%d")

# Arquivo alvo
target_file = f"ecmwf_rain_tp_run_{run_date_str}_00Z_15d.grib2"

# Steps 0-144h a cada 3h, 150-360h a cada 6h
steps_all = list(range(0,145,3)) + list(range(150,361,6))
request_params = {
    "date": run_date_str,
    "time": 0,
    "step": steps_all,
    "param": "tp",
    "type": "fc",
    "levtype": "sfc",
    "stream": "oper",
    "target": target_file
}

print(f"\n📡 Tentando baixar ECMWF HRES {run_date_str} 00Z (15 dias)...")
try:
    if not os.path.exists(target_file):
        client.retrieve(**request_params)
        print(f"✅ Download concluído: {target_file}")
    else:
        print(f"⚠️ Arquivo já existe: {target_file}")
except Exception as e:
    # fallback 00Z do dia anterior
    date_prev = date_run - timedelta(days=1)
    run_date_prev_str = date_prev.strftime("%Y%m%d")
    target_file_prev = f"ecmwf_rain_tp_run_{run_date_prev_str}_00Z_15d.grib2"
    request_params.update({"date": run_date_prev_str, "time": 0, "target": target_file_prev})
    print(f"⚠️ Falha no download 00Z. Tentando fallback 00Z do dia anterior ({run_date_prev_str})...")
    try:
        client.retrieve(**request_params)
        print(f"✅ Download alternativo concluído: {target_file_prev}")
    except Exception as e2:
        print("❌ Erro no download alternativo:", e2)
        raise SystemExit("Falha ao obter arquivo ECMWF. Pare e verifique conexão/parametros.")

# ==============================
# 5. Abrir GRIB2
# ==============================
print("\n📂 Abrindo arquivo GRIB2...")
ds = xr.open_dataset(target_file, engine="cfgrib", filter_by_keys={"typeOfLevel": "surface"})
tp_mm = ds["tp"] * 1000.0
run_time = pd.to_datetime(tp_mm["time"].item()).to_pydatetime()
print(f"Rodada detectada: {run_time:%Y-%m-%d %H:%MZ}")

# ==============================
# 6. Precipitação diária 24h (UTC-3)
# ==============================
utc_offset = -3
n_days = 15
daily = []

# Step times cumulativos
step_times = run_time + pd.to_timedelta(tp_mm.step.values, unit='h')

for day in range(n_days):
    start_br = datetime(run_time.year, run_time.month, run_time.day) + timedelta(days=day)
    end_br   = start_br + timedelta(hours=24)

    # Converter para UTC
    start_br_utc = start_br - timedelta(hours=utc_offset)
    end_br_utc   = end_br - timedelta(hours=utc_offset)

    # Encontrar indices mais próximos
    step_start = np.argmin(np.abs(step_times - start_br_utc))
    step_end   = np.argmin(np.abs(step_times - end_br_utc))

    if day == 0:
        data_24h = tp_mm.isel(step=step_end)
    else:
        data_24h = tp_mm.isel(step=step_end) - tp_mm.isel(step=step_start)

    daily.append({"data": data_24h, "start": start_br, "end": end_br})

# ==============================
# 7. Gerar mapas diários (com títulos corretos)
# ==============================
print("\n🗺️ Gerando mapas diários...")
for idx, item in enumerate(daily):
    daynum = idx + 1
    rain = item["data"]
    start = item["start"]
    end = item["end"]

    fig = plt.figure(figsize=(10,8))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_extent(extent, crs=ccrs.PlateCarree())
    ax.coastlines(resolution="10m", linewidth=0.8, color='black')
    ax.add_feature(NaturalEarthFeature("cultural", "admin_0_countries","50m", edgecolor="black", facecolor="none", linewidth=0.8))
    ax.add_feature(NaturalEarthFeature("cultural", "admin_1_states_provinces_lines","50m", edgecolor="black", facecolor="none", linewidth=0.8))
    ax.gridlines(draw_labels=False, linestyle="--", alpha=0.4)

    cf = rain.plot.contourf(ax=ax, transform=ccrs.PlateCarree(),
                            cmap=color_map, norm=norma, levels=nivels, extend="neither", add_colorbar=False)

    dia_semana = dias_semana_pt[start.strftime("%A")]
    ax.set_title(f"{start:%d-%m-%y} ({dia_semana})\nRodada ECMWF: {run_time:%d-%m-%Y %H:%MZ}", fontsize=11, weight="bold")

    cbar = plt.colorbar(cf, ax=ax, orientation="vertical", fraction=0.04, pad=0.02)
    cbar.set_ticks(tick_locs)
    cbar.set_ticklabels(tick_labels)
    cbar.set_label("Precipitação (mm/24h)")

    fname = os.path.join(out_dir, f"{daynum:02d}.png")
    plt.savefig(fname, dpi=300, bbox_inches="tight")
    plt.close(fig)
    print(f"✅ Salvo: {fname}")

# ==============================
# 8. Mapa acumulado 15 dias
# ==============================
accum_15d = sum([item["data"] for item in daily])
start_acc = daily[0]['start']
end_acc = daily[-1]['end']

fig = plt.figure(figsize=(10,8))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent(extent, crs=ccrs.PlateCarree())
ax.coastlines(resolution="10m", linewidth=0.8, color='black')
ax.add_feature(NaturalEarthFeature("cultural", "admin_0_countries","50m", edgecolor="black", facecolor="none", linewidth=0.8))
ax.add_feature(NaturalEarthFeature("cultural", "admin_1_states_provinces_lines","50m", edgecolor="black", facecolor="none", linewidth=0.8))
ax.gridlines(draw_labels=False, linestyle="--", alpha=0.4)

cf = accum_15d.plot.contourf(ax=ax, transform=ccrs.PlateCarree(),
                             cmap=color_map, norm=norma, levels=nivels, extend="neither", add_colorbar=False)
ax.set_title(f"Precipitação acumulada - 15 dias\nPeríodo: {start_acc:%d-%m} até {end_acc:%d-%m}\nRodada ECMWF: {run_time:%d-%m-%Y %HZ}", fontsize=12, weight="bold")

cbar = plt.colorbar(cf, ax=ax, orientation="vertical", fraction=0.04, pad=0.02)
cbar.set_ticks(tick_locs)
cbar.set_ticklabels(tick_labels)

fname_acc = os.path.join(out_dir, "acumulado-15-dias.png")
plt.savefig(fname_acc, dpi=600, bbox_inches="tight")
plt.close(fig)
print(f"✅ Salvo mapa acumulado 15 dias: {fname_acc}")


📡 Tentando baixar ECMWF HRES 20251029 00Z (15 dias)...
⚠️ Arquivo já existe: ecmwf_rain_tp_run_20251029_00Z_15d.grib2

📂 Abrindo arquivo GRIB2...
Rodada detectada: 2025-10-29 00:00Z

🗺️ Gerando mapas diários...
✅ Salvo: d:\Projetos\previsao-do-tempo\previsao\01.png
✅ Salvo: d:\Projetos\previsao-do-tempo\previsao\02.png
✅ Salvo: d:\Projetos\previsao-do-tempo\previsao\03.png
✅ Salvo: d:\Projetos\previsao-do-tempo\previsao\04.png
✅ Salvo: d:\Projetos\previsao-do-tempo\previsao\05.png
✅ Salvo: d:\Projetos\previsao-do-tempo\previsao\06.png
✅ Salvo: d:\Projetos\previsao-do-tempo\previsao\07.png
✅ Salvo: d:\Projetos\previsao-do-tempo\previsao\08.png
✅ Salvo: d:\Projetos\previsao-do-tempo\previsao\09.png
✅ Salvo: d:\Projetos\previsao-do-tempo\previsao\10.png
✅ Salvo: d:\Projetos\previsao-do-tempo\previsao\11.png
✅ Salvo: d:\Projetos\previsao-do-tempo\previsao\12.png
✅ Salvo: d:\Projetos\previsao-do-tempo\previsao\13.png
✅ Salvo: d:\Projetos\previsao-do-tempo\previsao\14.png
✅ Salvo: d:\Projet